#1. Predição de pagamento de empréstimos

Criação de modelos de classificação para inferir se um novo cliente que submeteu uma requisição de empréstimo pagará ou não a dívida com base nas características desse novo cliente.

Conjuntos de dados: credtrain.txt, credtest.txt

Número de variáveis independentes: 11

Variável dependente: "CLASSE", [0,1] onde, 0 não pagará a dívida e 1 pagará a dívida

Modelos utilizados: LogisticRegression, DecisionTree, RandomForest, KNeighbors, GradientBoosting

##1.1 Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import ConfusionMatrixDisplay



##1.2 Importação de dados

Importação dos arquivos de treino e teste disponibilizados no github através da biblioteca pandas

In [2]:
data_train_E1 = pd.read_table("https://raw.githubusercontent.com/RodrigoSFMachado/Data_Science/main/data/credtrain.txt", header= None)
data_test_E1 = pd.read_table("https://raw.githubusercontent.com/RodrigoSFMachado/Data_Science/main/data/credtest.txt", header= None)

##1.3 Pre-processamento de dados

Observando a estrutura dos dados de treinamento e teste

In [3]:
data_train_E1.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,0,360,0,313,9,52,0,25,48,0,1
1,0,0,350,1,468,10,65,0,33,6,0,1
2,0,0,1100,0,829,9,125,0,56,48,0,1


In [4]:
data_test_E1.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,2,500,1,618,10,85,0,36,6,0,0
1,1,0,813,0,552,4,119,0,43,48,119,1
2,3,0,350,0,488,12,66,0,43,0,0,1


Nomeando as variáveis para melhor leitura dos dados

In [5]:
colunas_E1 = ['ESCT', 'NDEP', 'RENDA', 'TIPOR', 'VBEM', 'NPARC', 'VPARC', 'TEL', 'IDADE', 'RESMS', 'ENTRADA', 'CLASSE']
data_train_E1.columns = colunas_E1
data_test_E1.columns = colunas_E1

In [6]:
data_train_E1.head(1)

,ESCT,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE
0,1,0,360,0,313,9,52,0,25,48,0,1


In [7]:
data_test_E1.head(1)

,ESCT,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE
0,0,2,500,1,618,10,85,0,36,6,0,0


Checando informações sobre os dados e verificando a tabela possui campos nulos

In [8]:
data_train_E1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   ESCT     1500 non-null   int64
 1   NDEP     1500 non-null   int64
 2   RENDA    1500 non-null   int64
 3   TIPOR    1500 non-null   int64
 4   VBEM     1500 non-null   int64
 5   NPARC    1500 non-null   int64
 6   VPARC    1500 non-null   int64
 7   TEL      1500 non-null   int64
 8   IDADE    1500 non-null   int64
 9   RESMS    1500 non-null   int64
 10  ENTRADA  1500 non-null   int64
 11  CLASSE   1500 non-null   int64
dtypes: int64(12)
memory usage: 140.8 KB


In [9]:
data_test_E1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   ESCT     577 non-null    int64
 1   NDEP     577 non-null    int64
 2   RENDA    577 non-null    int64
 3   TIPOR    577 non-null    int64
 4   VBEM     577 non-null    int64
 5   NPARC    577 non-null    int64
 6   VPARC    577 non-null    int64
 7   TEL      577 non-null    int64
 8   IDADE    577 non-null    int64
 9   RESMS    577 non-null    int64
 10  ENTRADA  577 non-null    int64
 11  CLASSE   577 non-null    int64
dtypes: int64(12)
memory usage: 54.2 KB


Separando a variável dependente (y) das variáveis independentes (X) nos dados de treino e teste

In [10]:
X_train_E1 = data_train_E1.drop('CLASSE', axis=1)
y_train_E1 = data_train_E1['CLASSE']
X_test_E1 = data_test_E1.drop('CLASSE', axis=1)
y_test_E1 = data_test_E1['CLASSE']

Realizando o escalonamento dos dados numéricos para mitigar discrepâncias entre as faixas de valores.

In [11]:
X_train_E1.head(1)

,ESCT,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA
0,1,0,360,0,313,9,52,0,25,48,0


In [12]:
scaler_E1 = StandardScaler()

def data_train_standard(X):

  X_standard = X[['RENDA', 'VBEM', 'NPARC', 'VPARC', 'IDADE', 'RESMS', 'ENTRADA']]
  X_standard = scaler_E1.fit_transform(X_standard)
  X_standard = pd.DataFrame(X_standard, columns= ('RENDA', 'VBEM', 'NPARC', 'VPARC', 'IDADE', 'RESMS', 'ENTRADA'))

  X_droped = X.drop(['RENDA', 'VBEM', 'NPARC', 'VPARC', 'IDADE', 'RESMS', 'ENTRADA'], axis=1)
  X_standard = X_droped.join(X_standard)

  return(X_standard)

def data_test_standard(X):

  X_standard = X[['RENDA', 'VBEM', 'NPARC', 'VPARC', 'IDADE', 'RESMS', 'ENTRADA']]
  X_standard = scaler_E1.transform(X_standard)
  X_standard = pd.DataFrame(X_standard, columns= ('RENDA', 'VBEM', 'NPARC', 'VPARC', 'IDADE', 'RESMS', 'ENTRADA'))

  X_droped = X.drop(['RENDA', 'VBEM', 'NPARC', 'VPARC', 'IDADE', 'RESMS', 'ENTRADA'], axis=1)
  X_standard = X_droped.join(X_standard)

  return(X_standard)


In [13]:
X_train_standard_final_E1 = data_train_standard(X_train_E1)
X_test_standard_final_E1 = data_test_standard(X_test_E1)

In [14]:
X_train_standard_final_E1.head(1)

,ESCT,NDEP,TIPOR,TEL,RENDA,VBEM,NPARC,VPARC,IDADE,RESMS,ENTRADA
0,1,0,0,0,-0.689892,-0.937611,0.168333,-0.798003,-1.22828,0.312878,-0.342137


A variável ESCT (Estado Civil) é tratada como uma variável categórica ordinal. Contudo, devido à ausência de uma ordem de grandeza entre os estados civis, optou-se por convertê-la em variável categórica nominal.








In [15]:
colunas_categoricas_E1 = ('ESCT_0', 'ESCT_1', 'ESCT_2', 'ESCT_3')

def data_categorical(X):

  X_categorical = X['ESCT']
  X_categorical = pd.get_dummies(X_categorical).astype(int)
  X_categorical.columns = colunas_categoricas_E1
  X_categorical = X.drop('ESCT',axis=1).join(X_categorical)
  return(X_categorical)


In [16]:
X_train_final_E1 = data_categorical(X_train_standard_final_E1)
X_test_final_E1 = data_categorical(X_test_standard_final_E1)

In [17]:
X_test_final_E1.head(3)

,NDEP,TIPOR,TEL,RENDA,VBEM,NPARC,VPARC,IDADE,RESMS,ENTRADA,ESCT_0,ESCT_1,ESCT_2,ESCT_3
0,2,1,0,-0.535236,0.210724,0.445500,-0.273914,-0.394811,-0.484925,-0.342137,1,0,0,0
1,0,0,0,-0.189468,-0.037768,-1.217504,0.266058,0.135577,0.312878,0.920112,0,1,0,0
2,0,0,0,-0.700939,-0.278730,0.999835,-0.575662,0.135577,-0.598897,-0.342137,0,0,0,1


##1.4 Criação/treinamento/avaliação do modelo

In [18]:
models_E1 = {
    'lr' : (LogisticRegression()),
    'dtc' : (DecisionTreeClassifier()),
    'rfc' : (RandomForestClassifier()),
    'gbc' : (GradientBoostingClassifier()),
    'knn' : (KNeighborsClassifier())
}

Treinamento dos modelos de classificação listados e avaliação através das métricas: Accuracy_score/Confusion_matrix/Classification_report

In [19]:
for model in models_E1.values():

  model.fit(X_train_final_E1, y_train_E1)
  y_pred_E1 = model.predict(X_test_final_E1)
  score = accuracy_score(y_test_E1 , y_pred_E1)
  confusion = confusion_matrix(y_test_E1, y_pred_E1)
  report = classification_report(y_test_E1, y_pred_E1)

  print(f'\nModel: {model} \n\nTest accuracy: {score*100:.2f}% \n\nConfusion_matrix: \n{confusion} \n\nClassification_report: \n{report}')
  print('================================================================')


Model: LogisticRegression() 

Test accuracy: 88.91% 

Confusion_matrix: 
[[290  16]
 [ 48 223]] 

Classification_report: 
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       306
           1       0.93      0.82      0.87       271

    accuracy                           0.89       577
   macro avg       0.90      0.89      0.89       577
weighted avg       0.89      0.89      0.89       577


Model: DecisionTreeClassifier() 

Test accuracy: 86.48% 

Confusion_matrix: 
[[261  45]
 [ 33 238]] 

Classification_report: 
              precision    recall  f1-score   support

           0       0.89      0.85      0.87       306
           1       0.84      0.88      0.86       271

    accuracy                           0.86       577
   macro avg       0.86      0.87      0.86       577
weighted avg       0.87      0.86      0.86       577


Model: RandomForestClassifier() 

Test accuracy: 89.43% 

Confusion_matrix: 
[[292  14]
 [ 47 2